MIT License

Copyright (c) Microsoft Corporation. All rights reserved.

This notebook is adapted from Microsoft Learning mslearn-dp100 

Copyright (c) 2021 PyLadies Amsterdam, Alyona Galyeva

# Create batch pipeline

## Connect to your workspace

In [1]:
from azureml.core import Workspace
ws = Workspace.from_config()

## Provision inference compute

We'll need a compute context for the pipeline, so we'll use the following code to specify an Azure Machine Learning compute cluster (it will be created if it doesn't already exist).

Important: Cluster names must be globally unique names between 2 to 16 characters in length. Valid characters are letters, digits, and the - character.

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "mlopsbootcamp"

try:
    # Check for existing compute target
    inference_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS2_v2', max_nodes=2)
        inference_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        inference_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Creating............
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


Note: Compute instances and clusters are based on standard Azure virtual machine images. For this exercise, the Standard_DS11_v2 image is recommended to achieve the optimal balance of cost and performance. If your subscription has a quota that does not include this image, choose an alternative image; but bear in mind that a larger image may incur higher cost and a smaller image may not be sufficient to complete the tasks. Alternatively, ask your Azure administrator to extend your quota.

## Create a pipeline for batch inferencing

Now we're ready to define the pipeline we'll use for batch inferencing. Our pipeline will need Python code to perform the batch inferencing, so let's create a folder where we can keep all the files used by the pipeline:

In [3]:
import os
# Create a folder for the experiment files
experiment_folder = 'batch_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

batch_pipeline


In [8]:
# let's check what models are registered in our workspace and get a path to our model of choice
from azureml.core import Model
model_list = Model.list(ws)
model_path = Model.get_model_path('linear_regression', _workspace=ws)
print(model_list, model_path)

[Model(workspace=Workspace.create(name='mlops', subscription_id='ef7a4699-23c4-4d90-8b71-e5d1e63f9154', resource_group='mlops_bootcamp'), name=linear_regression, id=linear_regression:1, version=1, tags={}, properties={})] azureml-models\linear_regression\1\linear_regression.pkl


In [9]:
# let's load our model and take a look what's inside
import joblib
model = joblib.load(model_path)
model


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('encoder',
                                                  OneHotEncoder(sparse=False),
                                                  [2, 3, 4])])),
                ('rfecv',
                 RFECV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                       estimator=LinearRegression(), n_jobs=-1,
                       scoring='neg_mean_squared_error', verbose=2))])

In [10]:
mini_batch = list()
for (dirpath, dirnames, filenames) in os.walk("batch-data"):
    mini_batch += [os.path.join(dirpath, file) for file in filenames]
for elem in mini_batch:
    print(elem)

batch-data\1.csv
batch-data\10.csv
batch-data\100.csv
batch-data\101.csv
batch-data\102.csv
batch-data\103.csv
batch-data\104.csv
batch-data\105.csv
batch-data\106.csv
batch-data\107.csv
batch-data\108.csv
batch-data\109.csv
batch-data\11.csv
batch-data\110.csv
batch-data\111.csv
batch-data\112.csv
batch-data\113.csv
batch-data\114.csv
batch-data\115.csv
batch-data\116.csv
batch-data\117.csv
batch-data\118.csv
batch-data\119.csv
batch-data\12.csv
batch-data\120.csv
batch-data\121.csv
batch-data\122.csv
batch-data\123.csv
batch-data\124.csv
batch-data\125.csv
batch-data\126.csv
batch-data\127.csv
batch-data\128.csv
batch-data\129.csv
batch-data\13.csv
batch-data\130.csv
batch-data\131.csv
batch-data\132.csv
batch-data\133.csv
batch-data\134.csv
batch-data\135.csv
batch-data\136.csv
batch-data\137.csv
batch-data\138.csv
batch-data\139.csv
batch-data\14.csv
batch-data\140.csv
batch-data\141.csv
batch-data\142.csv
batch-data\143.csv
batch-data\144.csv
batch-data\145.csv
batch-data\146.csv


In [11]:
import numpy as np
def run(mini_batch):
    # This runs for each batch
    resultList = []

    # process each file in the batch
    for f in mini_batch:
        # Read comma-delimited data into an array
        data = np.genfromtxt(f, delimiter=',')
        # Reshape into a 2-dimensional array for model input
        prediction = model.predict(data.reshape(1, -1))
        # Append prediction to results
        resultList.append("{}: {}".format(os.path.basename(f), prediction[0]))
    return resultList

In [12]:
result = run(mini_batch)
result

['1.csv: 5755.163077175251',
 '10.csv: 7322.167496592862',
 '100.csv: 6116.004871469763',
 '101.csv: 6156.092754264967',
 '102.csv: 6294.968788850575',
 '103.csv: 6782.238254913661',
 '104.csv: 7326.4008013360335',
 '105.csv: 7730.640952776846',
 '106.csv: 7647.589687514605',
 '107.csv: 8235.684934829878',
 '108.csv: 8638.614801197713',
 '109.csv: 8831.621426435839',
 '11.csv: 7611.775571331817',
 '110.csv: 8926.95040271835',
 '111.csv: 8852.551606579724',
 '112.csv: 8894.393962729115',
 '113.csv: 8935.391041913888',
 '114.csv: 8828.09936453811',
 '115.csv: 8485.005655694782',
 '116.csv: 8322.560818863683',
 '117.csv: 8041.131147118628',
 '118.csv: 7829.522731946653',
 '119.csv: 7627.891908286288',
 '12.csv: 7895.639868551175',
 '120.csv: 7179.049536344058',
 '121.csv: 6863.960961958594',
 '122.csv: 6576.478453586541',
 '123.csv: 6323.083432511578',
 '124.csv: 6216.794025926226',
 '125.csv: 6111.832799093078',
 '126.csv: 6214.011215538499',
 '127.csv: 6309.964012929901',
 '128.csv: 614

Now we'll create a Python script to do the actual work, and save it in the pipeline folder:

In [13]:
%%writefile $experiment_folder/score.py
# windows users
# %%writefile $experiment_folder\score.py

import os
import numpy as np
from azureml.core import Model
import joblib

def init():
    # Runs when the pipeline step is initialized
    global model

    # load the model
    model_path = Model.get_model_path('linear_regression')
    model = joblib.load(model_path)

def run(mini_batch):
    # This runs for each batch
    resultList = []

    # process each file in the batch
    for f in mini_batch:
        # Read comma-delimited data into an array
        data = np.genfromtxt(f, delimiter=',')
        # Reshape into a 2-dimensional array for model input
        prediction = model.predict(data.reshape(1, -1))
        # Append prediction to results
        resultList.append("{}: {}".format(os.path.basename(f), prediction[0]))
    return resultList

Writing batch_pipeline/score.py


The pipeline will need an environment in which to run, so we'll create a Conda specification that includes the packages that the code uses.

In [14]:
%%writefile $experiment_folder/batch_environment.yml
# windows users
# %%writefile $experiment_folder\batch_environment.yml
name: batch_environment
dependencies:
- python=3.8
- numpy
- pandas
- scikit-learn
- pip:
  - azureml-core

Writing batch_pipeline/batch_environment.yml


Next we'll define a run context that includes the Conda environment.

In [15]:
from azureml.core import Environment
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# Create an Environment for the experiment
batch_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/batch_environment.yml")
batch_env.docker.base_image = DEFAULT_CPU_IMAGE
print('Configuration ready.')

Configuration ready.


You're going to use a pipeline to run the batch prediction script, generate predictions from the input data, and save the results as a text file in the output folder. To do this, you can use a ParallelRunStep, which enables the batch data to be processed in parallel and the results collated in a single output file named parallel_run_step.txt.

In [18]:
from datetime import datetime

from azureml.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.data import OutputFileDatasetConfig
from azureml.core.runconfig import DockerConfiguration

# # Get the batch dataset for input
batch_data_set = ws.datasets['batch-data']

# Set the output location
default_ds = ws.get_default_datastore()
output_dir = OutputFileDatasetConfig(name='inferences')

# Define the parallel run step step configuration
parallel_run_config = ParallelRunConfig(
    source_directory=experiment_folder,
    entry_script="score.py",
    mini_batch_size="5",
    error_threshold=10,
    output_action="append_row",
    environment=batch_env,
    compute_target=inference_cluster,
    node_count=2)

parallel_step_name = "batchscoring-" + datetime.now().strftime("%Y%m%d%H%M")

# Create the parallel run step
parallelrun_step = ParallelRunStep(
    name=parallel_step_name,
    parallel_run_config=parallel_run_config,
    inputs=[batch_data_set.as_named_input('batch_data')],
    output=output_dir,
    arguments=[],
    allow_reuse=True
)

print('Steps defined')

Steps defined


C:\Users\madmin\.conda\envs\environment\lib\site-packages\azureml\pipeline\core\_parallel_run_step_base.py:591: UserWarning: 
ParallelRunStep requires azureml-dataset-runtime[fuse] for file dataset.
Please add relevant package in CondaDependencies.
  warnings.warn(


Now it's time to put the step into a pipeline, and run it.

In [19]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline

# Create the pipeline
pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])

# Run the pipeline as an experiment
pipeline_run = Experiment(ws, 'nyc-energy-demand-batch').submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)

Created step batchscoring-202107292226 [2ad45cf8][e66ec14f-d422-4fbc-a331-f819cd647bcb], (This step will run and generate new outputs)
Submitted PipelineRun 76cc9437-e4e8-42a8-bb5a-1e2d12c216d8
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/76cc9437-e4e8-42a8-bb5a-1e2d12c216d8?wsid=/subscriptions/ef7a4699-23c4-4d90-8b71-e5d1e63f9154/resourcegroups/mlops_bootcamp/workspaces/mlops&tid=68f8eea7-5130-4b29-9992-e2106b0299b5
PipelineRunId: 76cc9437-e4e8-42a8-bb5a-1e2d12c216d8
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/76cc9437-e4e8-42a8-bb5a-1e2d12c216d8?wsid=/subscriptions/ef7a4699-23c4-4d90-8b71-e5d1e63f9154/resourcegroups/mlops_bootcamp/workspaces/mlops&tid=68f8eea7-5130-4b29-9992-e2106b0299b5
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 21f37741-f9ed-4fb9-ad4e-cf55b955087a
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/21f37741-f9ed-4fb9-ad4e-cf55b955087a?wsid=/subscriptions/ef7a4699-23c4-4d90-8b71-e5d1e

'Finished'

When the pipeline has finished running, the resulting predictions will have been saved in the outputs of the experiment associated with the first (and only) step in the pipeline. You can retrieve it as follows:

In [20]:
import pandas as pd
import shutil

# Remove the local results folder if left over from a previous run
shutil.rmtree('batch-results', ignore_errors=True)

# Get the run for the first step and download its output
prediction_run = next(pipeline_run.get_children())
prediction_output = prediction_run.get_output_data('inferences')
prediction_output.download(local_path='batch-results')

# Traverse the folder hierarchy and find the results file
for root, dirs, files in os.walk('batch-results'):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# cleanup output format
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]

# Display the first 20 results
df.head(20)

,File,Prediction
0,1.csv,5755.163077
1,10.csv,7322.167497
2,100.csv,6116.004871
3,101.csv,6156.092754
4,102.csv,6294.968789
5,103.csv,6782.238255
6,104.csv,7326.400801
7,105.csv,7730.640953
8,106.csv,7647.589688
9,107.csv,8235.684935


## Publish the Pipeline and use its REST Interface

Now that you have a working pipeline for batch inferencing, you can publish it and use a REST endpoint to run it from an application.

In [21]:
published_pipeline = pipeline_run.publish_pipeline(name='Linear_regression_batch_prediction_pipeline',
                                                   description='Batch scoring using linear regression model',
                                                   version='1.0')

published_pipeline

Name,Id,Status,Endpoint
Linear_regression_batch_prediction_pipeline,99322a55-ca44-4d4e-b63d-4d1bad6200fb,Active,REST Endpoint


Note that the published pipeline has an endpoint, which you can see in the Azure portal. You can also find it as a property of the published pipeline object:

In [22]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

https://westeurope.api.azureml.ms/pipelines/v1.0/subscriptions/ef7a4699-23c4-4d90-8b71-e5d1e63f9154/resourceGroups/mlops_bootcamp/providers/Microsoft.MachineLearningServices/workspaces/mlops/PipelineRuns/PipelineSubmit/99322a55-ca44-4d4e-b63d-4d1bad6200fb


To use the endpoint, client applications need to make a REST call over HTTP. This request must be authenticated, so an authorization header is required. To test this out, we'll use the authorization header from your current connection to your Azure workspace, which you can get using the following code:

Note: A real application would require a service principal with which to be authenticated.

In [23]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print('Authentication header ready.')

Authentication header ready.


Once published, you can use this endpoint to initiate a batch inferencing job, as shown in the following example code:

In [24]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "nyc-energy-demand-batch"})
run_id = response.json()["Id"]
run_id

'd7037e38-9c63-4049-87dd-76a6b565bada'

You can also schedule the published pipeline to have it run automatically, as shown in the following example code:

In [25]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

weekly = ScheduleRecurrence(frequency='Week', interval=1)
pipeline_schedule = Schedule.create(ws, name='Weekly Predictions',
                                        description='batch inferencing',
                                        pipeline_id=published_pipeline.id,
                                        experiment_name='Batch_Prediction',
                                        recurrence=weekly)

IMPORTANT: Remove inference cluster if you do not plan to work on exercises immediately!!!